In [1]:
import numpy as np
import sys
sys.path.append('../solution')
from test_cases import TEST_CASES
from world import calc_score, MOVES, roll_coordinates, get_neighbours, apply_to_map, find_disjoint_components, update_direction
from solver import HC_1NB_CELLS, find_variants_for_mc
import time

In [2]:
_map, robots = TEST_CASES[29]
print(_map)
_map = np.array([list(l) for l in _map.split('\n')])

#........L........#
#........R........#
#........L........#
#........R........#
#DUDUDUDU#UDUDUDUD#
#........R........#
#........L........#
#........R........#
#........L........#
###################


In [12]:
variants = find_variants_for_mc(_map)
hot_points = {}
for v in variants:
    hot_points[v[0]] = v[1]
arrows = {}

In [13]:
def move_till_hp_or_death(w, x, y, d, _map, visited_states, arrows, hot_points):
    sp = (x, y)
    arrows[sp] = w
    x, y, d, is_live, new_visited_states = move_till_hp_or_death2(x, y, d, _map,  visited_states, arrows, hot_points)
    del arrows[sp]
    return w, x, y, d, is_live, new_visited_states


def move_till_hp_or_death2(x, y, d, _map, visited_states, arrows, hot_points):
    new_visited_states = set()
    d = update_direction(_map, x, y, d, arrows)
    is_live = True
    while is_live and (x, y) not in hot_points:
        new_visited_states.add((x, y, d))
        x += MOVES[d][0]
        y += MOVES[d][1]
        x, y = roll_coordinates(x, y)
        d = update_direction(_map, x, y, d, arrows)
        is_live = _map[y, x] != '#' and (x, y, d) not in visited_states and (x, y, d) not in new_visited_states
    return x, y, d, is_live, new_visited_states

In [14]:
is_live = []
states = []
visited = []
for x, y, d in robots:
    states.append((x, y, d))
    visited.append(set())
    is_live.append(True)

In [15]:
while any(is_live):
    for i in range(len(robots)):
        x, y, d = states[i]
        sp = (x, y)
        if sp in hot_points:            
            hot_point = hot_points[sp]
            del hot_points[sp]
            ways = []
            for w in hot_point:
                if w == '':
                    continue
                ways.append(move_till_hp_or_death(w, x, y, d, _map, visited[i], arrows, hot_points))
            w, x, y, d, is_live[i], new_visited_states = max(ways, key=lambda x: (x[4], len(x[5])))
            arrows[sp] = w    
        else:
            x, y, d, is_live[i], new_visited_states = move_till_hp_or_death2(x, y, d, _map, visited[i], arrows, hot_points)
        visited[i] = visited[i].union(new_visited_states)
        states[i] = (x, y, d)

In [16]:
txt = ''
for e in arrows:
    txt += f'{e[0]} {e[1]} {arrows[e]} '
txt

'1 0 D 17 0 D 1 8 U 17 8 U 1 1 R 17 1 D 1 7 R 17 7 U 17 2 L 17 6 L 1 2 D 1 6 U 1 3 R 1 5 R 17 3 D 17 5 L '